# 데이터 2차 전처리 ( Serial number 가 중복으로 작성된 건지 확인)

In [14]:
import pandas as pd
import numpy as np

path = "C:/Users/user/Desktop/Survival_Data/MAIN/cancer_busan.xlsx"
df = pd.read_excel(path)

In [15]:
df.head()

,SERIAL,sex,age,job,fdx,year,visitpath,visitpath_,later,mcode,...,i_adrcode,tx,seercode,stagecode,stagedesc,metacode1,death,sain,DONG_F,WEIGHT
0,RBS000989,1,68,NaN,1999,1999,NaN,NaN,NaN,80003,...,20.0,0000,NaN,NaN,Unknown,NaN,20010102.0,C169,동광동,1.0
1,A06006581,1,49,3.0,1999,1999,NaN,NaN,NaN,80413,...,20.0,01100,7,TNM,TNM1/4,NaN,20071208.0,C349,만덕1동,1.0
2,RBS000286,2,73,99.0,1999,1999,NaN,NaN,NaN,80003,...,20.0,0000,NaN,NaN,Unknown,NaN,20010502.0,C55,장안읍,1.0
3,G05114813,2,69,14.0,1999,1999,NaN,NaN,NaN,80003,...,20.0,00000,9,NaN,NaN,NaN,20180619.0,N11,당감1동,1.0
4,G03077260,1,63,99.0,1999,1999,NaN,NaN,NaN,80003,...,20.0,00000,9,NaN,NaN,NaN,20050410.0,C169,주례1동,1.0


### 중복된 SERIAL number 존재

In [16]:
print(df["SERIAL"].value_counts())

SERIAL
G16011541    8
A15553749    8
A18761824    8
A07105025    8
A01064056    8
            ..
G10010514    1
A11500216    1
G10011505    1
A10621094    1
A13342777    1
Name: count, Length: 364386, dtype: int64


### SERIAL 컬럼 ID 부여 로직
1. unique() 값들을 도출
2. unique() 값들에 대한 index 넘버 생성과 동시에 딕셔너리에 집어넣ㄱ
3. SERIAL 컬럼에 해당 딕셔너리들을 매핑

In [17]:
unique_serials = df["SERIAL"].unique()
# print(unique_serials)
serial_to_id = {serial: idx for idx, serial in enumerate(unique_serials, start=1)}
df["SERIAL_ID"] = df["SERIAL"].map(serial_to_id)

In [18]:
df["SERIAL_ID"].value_counts()

SERIAL_ID
242258    8
231487    8
286171    8
83967     8
16474     8
         ..
125735    1
125734    1
125733    1
125732    1
182194    1
Name: count, Length: 364386, dtype: int64

In [19]:
df["death"].value_counts()

death
11111111.0    81
20190313.0    65
20200824.0    61
20170902.0    60
20210426.0    60
              ..
19990123.0     1
19990506.0     1
19990128.0     1
19990220.0     1
19990127.0     1
Name: count, Length: 9063, dtype: int64

### 추가 전처리 과정

1. 생존기간 
    * [death] 에서 111111 로 적혀있는 값들은 사망일과 초진일이 맞지 않는경우 임으로 삭제.
2. TREATMENT 더미변수화
    * U =0 으로 처리
    * zfill=5, 왼쪽에서부터 0으로 채움
    * 예시 : U110 -> 00110
3. EVENT INDICATOR 생성(사망 =1 / NULL or inf() = 0)

### 사망 컬럼 "111111.0" 값 제거

In [20]:
# 11111111.0 값을 가진 행 제거
df = df[df["death"] != 11111111.0]

In [21]:
df

,SERIAL,sex,age,job,fdx,year,visitpath,visitpath_,later,mcode,...,tx,seercode,stagecode,stagedesc,metacode1,death,sain,DONG_F,WEIGHT,SERIAL_ID
0,RBS000989,1,68,NaN,1999,1999,NaN,NaN,NaN,80003,...,0000,NaN,NaN,Unknown,NaN,20010102.0,C169,동광동,1.000000,1
1,A06006581,1,49,3.0,1999,1999,NaN,NaN,NaN,80413,...,01100,7,TNM,TNM1/4,NaN,20071208.0,C349,만덕1동,1.000000,2
2,RBS000286,2,73,99.0,1999,1999,NaN,NaN,NaN,80003,...,0000,NaN,NaN,Unknown,NaN,20010502.0,C55,장안읍,1.000000,3
3,G05114813,2,69,14.0,1999,1999,NaN,NaN,NaN,80003,...,00000,9,NaN,NaN,NaN,20180619.0,N11,당감1동,1.000000,4
4,G03077260,1,63,99.0,1999,1999,NaN,NaN,NaN,80003,...,00000,9,NaN,NaN,NaN,20050410.0,C169,주례1동,1.000000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414337,A22852784,2,65,14.0,20211231,2021,1.0,NaN,2.0,82533,...,11000,1,TNM,pT4N0M0,NaN,NaN,NaN,대저2동,1.000000,364384
414338,A22642674,2,38,99.0,20211231,2021,1.0,NaN,0.0,80772,...,10000,0,NaN,NaN,NaN,NaN,NaN,재송1동,0.679095,364385
414339,A22642674,2,38,99.0,20211231,2021,1.0,NaN,0.0,80772,...,10000,0,NaN,NaN,NaN,NaN,NaN,재송2동,0.320905,364385
414340,A22684733,1,56,99.0,20211231,2021,3.0,NaN,0.0,81403,...,11000,4,NaN,NaN,NaN,NaN,NaN,반송1동,0.403499,364386


### 생존기간 처리

In [23]:
import numpy as np

df_transformed = df.copy()

# evnet_indicator 생성
# null 과 inf 값들은 0으로 처리, 그렇지 않으면은 1로 처리
df_transformed["event_inc"] = np.where(
    df_transformed["death"].isna() | (df_transformed["death"] == float("inf")), 0, 1
)

# death 에 존재하는 null 값과 inf 값들은 20240101로 대체
df_transformed["death"] = (
    df_transformed["death"].replace([np.nan, float("inf")], 20240101).astype(int)
)

In [24]:
# fdx 컬럼에서 4글자인경우 (연도 : 1999, 2014만 입력되어있는 경우) 에는 해당 값은 추적이 되지 않을걸로 보고 삭제
df_transformed = df_transformed[df_transformed["fdx"].astype(str).str.len() != 4]

# 만약 8글자(19990101 과 같이 연월일인 경우) 6글자(연월) 로 변경
df_transformed["fdx"] = (
    df_transformed["fdx"].astype(str).apply(lambda x: x[:6] if len(x) == 8 else x)
)

In [25]:
# fdx 컬럼에서 4글자인경우 (연도 : 1999, 2014만 입력되어있는 경우) 에는 해당 값은 추적이 되지 않을걸로 보고 삭제
df_transformed = df_transformed[df_transformed["death"].astype(str).str.len() != 4]

# 만약 8글자(19990101 과 같이 연월일인 경우) 6글자(연월) 로 변경
df_transformed["death"] = (
    df_transformed["death"].astype(str).apply(lambda x: x[:6] if len(x) == 8 else x)
)

## fdx 와 death 컬럼들의 값들 재정제

1. "201500" 과 같이 "월" 단위가 1,2,3,4,5,6,7,8,9,10,11,12 에 포함되어있지 않으면 삭제

2. 100000 과 같이 "연도"가 1999~202401 사이가 아닌 경우도 삭제

In [26]:
# fdx와 death 값을 문자열로 변환
df_transformed["fdx_str"] = df_transformed["fdx"].astype(str)
df_transformed["death_str"] = df_transformed["death"].astype(str)

In [27]:
# 월 값 추출: 문자열의 뒤 두 자리
df_transformed["fdx_month"] = pd.to_numeric(
    df_transformed["fdx_str"].str[-2:], errors="coerce"
)
df_transformed["death_month"] = pd.to_numeric(
    df_transformed["death_str"].str[-2:], errors="coerce"
)

In [28]:
print(df_transformed["fdx_month"])

66         1
67         1
68         1
69         1
70         1
          ..
414337    12
414338    12
414339    12
414340    12
414341    12
Name: fdx_month, Length: 412357, dtype: int64


In [29]:
# 조건 1: 월 값이 1~12 사이에 있는지 필터링
df_transformed = df_transformed[
    (df_transformed["fdx_month"] >= 1) & (df_transformed["fdx_month"] <= 12)
]
df_transformed = df_transformed[
    (df_transformed["death_month"] >= 1) & (df_transformed["death_month"] <= 12)
]

In [30]:
# fdx와 death 값을 정수로 변환
df_transformed["fdx_int"] = pd.to_numeric(df_transformed["fdx"], errors="coerce")
df_transformed["death_int"] = pd.to_numeric(df_transformed["death"], errors="coerce")

In [31]:
# 조건 2: 연월 값이 199901 ~ 202401 사이에 있는지 필터링
df_transformed = df_transformed[
    (df_transformed["fdx_int"] >= 199901) & (df_transformed["fdx_int"] <= 202401)
]
df_transformed = df_transformed[
    (df_transformed["death_int"] >= 199901) & (df_transformed["death_int"] <= 202401)
]

In [32]:
# fdx와 death를 날짜 형식으로 변환
df_transformed["fdx_date"] = pd.to_datetime(
    df_transformed["fdx"].astype(str), format="%Y%m", errors="coerce"
)
df_transformed["death_date"] = pd.to_datetime(
    df_transformed["death"].astype(str), format="%Y%m", errors="coerce"
)

In [33]:
# 기간(월 단위) 계산
df_transformed["stime"] = (
    (df_transformed["death_date"].dt.year - df_transformed["fdx_date"].dt.year) * 12
    + (df_transformed["death_date"].dt.month - df_transformed["fdx_date"].dt.month)
).abs()  # 절대값으로 처리 (음수 방지)

### 치료 전처리

In [34]:
def create_tx_variables(tx_value):
    """
    tx_value를 입력받아 5자리 문자열로 변환하며, 주어진 규칙을 적용합니다:
    1. null(NaN) -> '00000'
    2. 'U'를 '0'으로 변환 (예: "U1UU" -> "0100")
    3. 5자리로 변환, 왼쪽에 '0' 추가 (예: "0100" -> "00100")
    4. 각 자릿수를 분리하여 반환

    Args:
        tx_value: tx 열의 개별 값 (float 또는 str)

    Returns:
        tx_1, tx_2, tx_3, tx_4, tx_5: 각 자릿수를 나타내는 값들
    """
    # 1. Null 값 확인 및 처리
    if pd.isna(tx_value):
        tx_str = "00000"
    else:
        # 2. 문자열로 변환 후 'U'를 '0'으로 변환
        tx_str = str(tx_value).replace("U", "0")

        # 숫자가 아닌 문자를 제거 (예: '.', 공백 등)
        tx_str = "".join(filter(str.isdigit, tx_str))

        # 3. 5자리로 맞춤
        tx_str = tx_str.zfill(5)

    # 각 자리수 변환
    try:
        tx_1 = int(tx_str[0])
        tx_2 = int(tx_str[1])
        tx_3 = int(tx_str[2])
        tx_4 = int(tx_str[3])
        tx_5 = int(tx_str[4])
    except ValueError:
        # 숫자로 변환할 수 없는 값이 있을 경우 기본값 '00000' 사용
        tx_1, tx_2, tx_3, tx_4, tx_5 = 0, 0, 0, 0, 0

    return tx_1, tx_2, tx_3, tx_4, tx_5


# tx 컬럼에 대해 함수 적용하여 새로운 컬럼 추가
df_transformed[["tx_1", "tx_2", "tx_3", "tx_4", "tx_5"]] = (
    df_transformed["tx"].apply(create_tx_variables).apply(pd.Series)
)

           SERIAL  sex  age   job     fdx  year  visitpath visitpath_  later  \
66      A99060877    2   49  13.0  199901  1999        NaN        NaN    NaN   
67      RBS000664    2   83  99.0  199901  1999        NaN        NaN    NaN   
68      A99048976    1   55   2.0  199901  1999        NaN        NaN    NaN   
69      RBS068645    1   78  99.0  199901  1999        NaN        NaN    NaN   
70      A00034919    1   60   4.0  199901  1999        NaN        NaN    NaN   
...           ...  ...  ...   ...     ...   ...        ...        ...    ...   
414337  A22852784    2   65  14.0  202112  2021        1.0        NaN    2.0   
414338  A22642674    2   38  99.0  202112  2021        1.0        NaN    0.0   
414339  A22642674    2   38  99.0  202112  2021        1.0        NaN    0.0   
414340  A22684733    1   56  99.0  202112  2021        3.0        NaN    0.0   
414341  A22684733    1   56  99.0  202112  2021        3.0        NaN    0.0   

        mcode  ...  fdx_int  death_int 

### 사용하지 않는 컬럼 제거

### 사용하지 않는 컬럼 목록
['stagecode','stagedesc','fdx','death','age_group',
'bunhwado','mcode','metacode1','death','sain','tx']

In [35]:
df_transformed.head()

,SERIAL,sex,age,job,fdx,year,visitpath,visitpath_,later,mcode,...,fdx_int,death_int,fdx_date,death_date,stime,tx_1,tx_2,tx_3,tx_4,tx_5
66,A99060877,2,49,13.0,199901,1999,NaN,NaN,NaN,81403,...,199901,199907,1999-01-01,1999-07-01,6,0,0,1,0,0
67,RBS000664,2,83,99.0,199901,1999,NaN,NaN,NaN,80003,...,199901,200105,1999-01-01,2001-05-01,28,0,0,0,0,0
68,A99048976,1,55,2.0,199901,1999,NaN,NaN,NaN,81403,...,199901,199911,1999-01-01,1999-11-01,10,0,1,0,1,0
69,RBS068645,1,78,99.0,199901,1999,NaN,NaN,NaN,80003,...,199901,199910,1999-01-01,1999-10-01,9,0,0,0,0,0
70,A00034919,1,60,4.0,199901,1999,NaN,NaN,NaN,81703,...,199901,200010,1999-01-01,2000-10-01,21,0,0,1,0,0


In [36]:
df_transformed.columns

Index(['SERIAL', 'sex', 'age', 'job', 'fdx', 'year', 'visitpath', 'visitpath_',
       'later', 'mcode', 'bunhwado', 'method', 'icd_10', 'adr1', 'adrcode',
       'i_adr1', 'i_adr2', 'bg', 'gu', 'dong', 'i_adrcode', 'tx', 'seercode',
       'stagecode', 'stagedesc', 'metacode1', 'death', 'sain', 'DONG_F',
       'WEIGHT', 'SERIAL_ID', 'event_inc', 'fdx_str', 'death_str', 'fdx_month',
       'death_month', 'fdx_int', 'death_int', 'fdx_date', 'death_date',
       'stime', 'tx_1', 'tx_2', 'tx_3', 'tx_4', 'tx_5'],
      dtype='object')

In [37]:
df_transformed = df_transformed.drop(
    [
        "tx",
        "stagecode",
        "stagedesc",
        "fdx",
        "death",

        "bunhwado",
        "mcode",
        "metacode1",
        "death",
        "sain",
        "i_adr1",
        "i_adr2",
        "bg",
        "gu",
        "dong",
        "i_adrcode",

        "visitpath",
        "visitpath_",
    ],
    axis=1,
)

In [40]:
df_transformed.columns

Index(['SERIAL', 'sex', 'age', 'job', 'year', 'later', 'method', 'icd_10',
       'adr1', 'adrcode', 'seercode', 'DONG_F', 'WEIGHT', 'SERIAL_ID',
       'event_inc', 'fdx_str', 'death_str', 'fdx_month', 'death_month',
       'fdx_int', 'death_int', 'fdx_date', 'death_date', 'stime', 'tx_1',
       'tx_2', 'tx_3', 'tx_4', 'tx_5'],
      dtype='object')

In [41]:
df_transformed = df_transformed.drop(
    [
        "SERIAL",
        "job",
        "year",
        "later",

        "method",
        "adr1",
        "adrcode",
        "DONG_F",
        "WEIGHT",
        "fdx_str",
        "death_str",

        "fdx_month",
        "death_month",
        "fdx_int",
        "death_int",
    ],
    axis=1,
)

In [43]:
df_transformed = df_transformed.drop(["fdx_date", "death_date"], axis=1)

In [45]:
df_transformed.index = range(1, len(df_transformed) + 1)

In [46]:
df_transformed.to_csv("C:/Users/user/Desktop/Survival_Data/FINAL.csv", index=True)

In [50]:
df = pd.read_csv("C:/Users/user/Desktop/Survival_Data/FINAL.csv", index_col=0)

C:\Users\user\AppData\Local\Temp\ipykernel_16324\2723769425.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv('C:/Users/user/Desktop/Survival_Data/FINAL.csv', index_col =0)


In [51]:
df

,sex,age,icd_10,seercode,SERIAL_ID,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5
1,2,49,C800,NaN,48,1,6,0,0,1,0,0
2,2,83,C349,NaN,49,1,28,0,0,0,0,0
3,1,55,C250,NaN,50,1,10,0,1,0,1,0
4,1,78,C169,NaN,51,1,9,0,0,0,0,0
5,1,60,C220,NaN,52,1,21,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
412351,2,65,C343,1.0,364384,0,25,1,1,0,0,0
412352,2,38,D069,0.0,364385,0,25,1,0,0,0,0
412353,2,38,D069,0.0,364385,0,25,1,0,0,0,0
412354,1,56,C182,4.0,364386,0,25,1,1,0,0,0


In [52]:
df = df[["SERIAL_ID"] + [col for col in df.columns if col != "SERIAL_ID"]]

In [58]:
df

,SERIAL_ID,sex,age,icd_10,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5
1,48,2,49,C800,NaN,1,6,0,0,1,0,0
2,49,2,83,C349,NaN,1,28,0,0,0,0,0
3,50,1,55,C250,NaN,1,10,0,1,0,1,0
4,51,1,78,C169,NaN,1,9,0,0,0,0,0
5,52,1,60,C220,NaN,1,21,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
412351,364384,2,65,C343,1.0,0,25,1,1,0,0,0
412352,364385,2,38,D069,0.0,0,25,1,0,0,0,0
412353,364385,2,38,D069,0.0,0,25,1,0,0,0,0
412354,364386,1,56,C182,4.0,0,25,1,1,0,0,0


In [59]:
df.to_csv("C:/Users/user/Desktop/FINAL.csv", index=False)

# sksurv.ensemble 알아보기